In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow_probability.substrates import numpy as tfp
from cmdstanpy import CmdStanModel, set_cmdstan_path
import arviz as az

In [2]:
X = pd.read_csv("../Datasets/covariates.csv")
Y = pd.read_csv("../Datasets/Dataset_180.csv")
Y['Count_180'] = pd.to_numeric(Y['Count_180']).astype('Int64')
X = pd.get_dummies(X ,columns = ['Type'] ,prefix = 'type', drop_first = True)
X.loc[X['Station'] == 10171, 'Quota'] = 10
rows_to_drop_Y = Y[Y['Count_180'].isna()].index

# Remove corresponding rows from dataset X
X = X.drop(index=rows_to_drop_Y)
Y = Y.drop(index=rows_to_drop_Y)

# If you want to reset the index after dropping rows
X = X.reset_index(drop=True)
Y = Y.reset_index(drop=True)
stations = X['Station']
years = X['Year'] - 2009
unique_values = sorted(set(stations))
mapping_dict = {value: index + 1 for index, value in enumerate(unique_values)}
transformed_stations = [mapping_dict[value] for value in stations]
X = X.drop(['Year','Station','Month'], axis  = 1)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)

In [19]:
theta = [0.5] * X_std.shape[1]

In [25]:
var_selection = """

data {
  int<lower=1> N; // Number of observations
  int<lower=1> P; // Covariate number
  int<lower=1> nyears;


  array[N] int<lower=0> y; // Count data
  matrix[N, P] X; // Predictor matrix
  
  real tau; //True variance
  real c; //Small variance
  array[P] real theta;
}

parameters {
  vector[P] beta; // Coefficients for predictors
  
  vector[P] gamma; //Binomial for counting that covariate or not
}

transformed parameters {
    vector[N] lambda;
    vector[N] fix_eff;
    vector[P] sigma; //Variance for predictor
    array[P] real<lower=0> vari;
    
    for (i in 1:P)
    {
        vari[i] = theta[i]*(1-theta[i]);

    }

    fix_eff = X * beta;
    lambda = exp(fix_eff);
    
    sigma[1:P] = gamma[1:P]*tau + (1-gamma[1:P])*c;
}

model {

  beta[1:P] ~ normal(0, sigma[1:P]);
  y[1:N] ~ poisson(lambda[1:N]);
  
  gamma[1:P] ~ normal(theta, vari);

}
generated quantities{
 vector[N] log_lik;
 for(i in 1:N){
   log_lik [ i ] = poisson_lpmf (y [ i ] |lambda[i]) ;
}
}

"""

stan_file = "./var_sel.stan"

with open(stan_file, "w") as f:
    print(var_selection, file=f)

variable_selection = CmdStanModel(stan_file=stan_file)

ValueError: Failed to get source info for Stan model 'C:\Users\leoma\OneDrive\Documents\PoliMi\Bayesian statistics\Bayesian_Project\main\Modello\var_sel.stan'. Console:
Semantic error in 'C:\Users\leoma\OneDrive\Documents\PoliMi\Bayesian statistics\Bayesian_Project\main\Modello\var_sel.stan', line 32, column 18 to column 49:
   -------------------------------------------------
    30:      for (i in 1:P)
    31:      {
    32:          vari[i] = max(theta[i]*(1-theta[i]), dum);
                           ^
    33:  
    34:      }
   -------------------------------------------------

Ill-typed arguments supplied to function 'max':
(real, real)
Available signatures:
(int, int) => int
  The first argument must be int but got real
(array[] int) => int
  Expected 1 arguments but found 2 arguments.
(array[] real) => real
  Expected 1 arguments but found 2 arguments.
(vector) => real
  Expected 1 arguments but found 2 arguments.
(row_vector) => real
  Expected 1 arguments but found 2 argumen

In [23]:
data = {
    "N": len(Y),
    "P": X_std.shape[1],
    "nyears": 13,
    "y": Y['Count_180'],
    "X": X_std,
    "tau" : 2,
    "c" : 0.1,
    "theta" : theta
}


fit = variable_selection.sample(data=data, chains=4, parallel_chains=4, 
    iter_warmup=300, iter_sampling=200)
posterior_az = az.from_cmdstanpy(fit)

11:20:29 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

11:20:29 - cmdstanpy - ERROR - Chain [4] error: error during processing Operation not permitted
11:20:29 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
11:20:29 - cmdstanpy - ERROR - Chain [2] error: error during processing Operation not permitted
11:20:29 - cmdstanpy - ERROR - Chain [3] error: error during processing Operation not permitted


11:20:29 - cmdstanpy - INFO - CmdStan done processing.


RuntimeError: Error during sampling:
Exception: normal_lpdf: Scale parameter[2] is -2.62546, but must be positive! (in 'var_sel.stan', line 43, column 2 to column 36)
	Exception: normal_lpdf: Scale parameter[1] is -0.0588674, but must be positive! (in 'var_sel.stan', line 43, column 2 to column 36)
	Exception: normal_lpdf: Scale parameter[1] is -1.31373, but must be positive! (in 'var_sel.stan', line 43, column 2 to column 36)
	Exception: normal_lpdf: Scale parameter[1] is -1.19763, but must be positive! (in 'var_sel.stan', line 43, column 2 to column 36)
	Exception: normal_lpdf: Scale parameter[2] is -2.67677, but must be positive! (in 'var_sel.stan', line 43, column 2 to column 36)
	Exception: normal_lpdf: Scale parameter[2] is -2.8433, but must be positive! (in 'var_sel.stan', line 43, column 2 to column 36)
	Exception: normal_lpdf: Scale parameter[5] is -1.09842, but must be positive! (in 'var_sel.stan', line 43, column 2 to column 36)
	Exception: normal_lpdf: Scale parameter[1] is